In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

time_epoch = 60
epochs_per_batch = 2

# hard coded nrows
df_all = pd.read_csv('../../../diploma/multi-source-syber-security-events/flows.txt', header=None, nrows=100000)

df_all.columns = ['time', 'duration', 'source computer', 'source port', 'destination computer', 
              'destination port', 'protocol', 'packet count', 'byte count']

df = df_all[df_all['time'] <= epochs_per_batch * time_epoch]

df.index = df['time']
df.drop(columns=['time'],inplace=True)
df.head()

c:\users\sotiris\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,duration,source computer,source port,destination computer,destination port,protocol,packet count,byte count
time,,,,,,,,
1,0,C1065,389,C3799,N10451,6,10,5323
1,0,C1423,N1136,C1707,N1,6,5,847
1,0,C1423,N1142,C1707,N1,6,5,847
1,0,C14909,N8191,C5720,2049,6,1,52
1,0,C14909,N8192,C5720,2049,6,1,52


In [2]:
from helper import group_scale_data, group_scale_data_batch

groupped_data, hosts, parameters = group_scale_data(df, size_of_bin_seconds=time_epoch,
                                                    doScale=True, scaler='log')

In [3]:
groupped_data.head()

,number of flows,mean(byte count),epoch,source computer
0,53,30,1,C1015
1110,1,1,1,C3444
1111,1,1,1,C1315
1112,1,1,1,C867
1114,1,1,1,C1304


In [4]:
from pyspark.sql import SQLContext


from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext.getOrCreate()
ssc = StreamingContext(sc, 1)
sqlCtx = SQLContext(sc)


dataframe = sqlCtx.createDataFrame(groupped_data.drop(['source computer'], axis=1))
dataframe.show()
dataframe.columns

+---------------+----------------+-----+
|number of flows|mean(byte count)|epoch|
+---------------+----------------+-----+
|             53|              30|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
|              1|               1|    1|
+---------------+----------------+-----+
only showing top

['number of flows', 'mean(byte count)', 'epoch']

In [8]:
#TODO features labels
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['number of flows', 'mean(byte count)'], outputCol='features')


In [9]:
output = assembler.transform(dataframe)

In [10]:
output.show()

+---------------+----------------+-----+-----------+
|number of flows|mean(byte count)|epoch|   features|
+---------------+----------------+-----+-----------+
|             53|              30|    1|[53.0,30.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0,1.0]|
|              1|               1|    1|  [1.0

In [11]:

train_data, test_data = output.randomSplit([0.7, 0.3])

In [12]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.clustering import StreamingKMeans


trainingData = train_data
testingData = test_data

trainingQueue = [trainingData]
testingQueue = [testingData]

trainingStream = ssc.queueStream(trainingQueue)
testingStream = ssc.queueStream(testingQueue)

# We create a model with random clusters and specify the number of clusters to find
model = StreamingKMeans(k=2, decayFactor=1.0).setRandomCenters(3, 1.0, 0)


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Sotiris\\AppData\\Local\\Temp\\spark-749cc7dd-d145-40d9-b37b-082dcce05758\\pyspark-5f7b867f-02be-4f29-be4d-0315f4c893cb\\tmp_iw0t2vi'

In [ ]:

# Now register the streams for training and testing and start the job,
# printing the predicted cluster assignments on new data points as they arrive.
model.trainOn(trainingStream)

result = model.predictOnValues(testingStream.map(lambda lp: (lp.label, lp.features)))
result.pprint()

ssc.start()
ssc.stop(stopSparkContext=True, stopGraceFully=True)